In [1]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
import pandas as pd

In [2]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
csv_path = './data.csv'
df = pd.read_csv(csv_path)
documents = df['question'].values


In [4]:
documents

array(['What is the step by step guide to invest in share market in india?',
       'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       'How can I increase the speed of my internet connection while using a VPN?',
       ..., 'What is one coin?',
       'What is the approx annual cost of living while studying in UIC Chicago, for an Indian student?',
       'What is like to have sex with cousin?'], dtype=object)

In [5]:
document_ids = range(len(documents))
document_ids

range(0, 404351)

In [11]:
qdrant = QdrantClient("localhost", port=6333)

In [ ]:
qdrant.recreate_collection(
    collection_name = "my_books",
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE,
    ),
)

In [ ]:
uploaded_points = []

for doc_id, question in zip(document_ids, documents):
    try:
        vectorized_question = encoder.encode(question).tolist()
        point = models.PointStruct(
            id=doc_id,
            vector=vectorized_question,
            payload={"question": question}
        )
        uploaded_points.append(point)
    except Exception as e:
        print(f"Error processing document {doc_id}: {e}")

qdrant.upload_points(
    collection_name="my_books",
    points=uploaded_points
)


In [20]:
hits = qdrant.search(
    collection_name="my_books",
    query_vector = encoder.encode("what is best way to invest in india").tolist(),
    limit=5
)

In [21]:
for hit in hits:
    print(hit.payload, "score: ", hit.score)

{'question': 'What are some of the best ways to invest money in India?'} score:  0.94016796
{'question': 'What is the best way to invest money in stocks in India?'} score:  0.9017929
{'question': 'What are best investment options in India?'} score:  0.89098483
{'question': 'How do I invest my money in India?'} score:  0.8877584
{'question': 'What is best investment option in India?'} score:  0.887447
